In [110]:
import urllib.request
import datetime
import time
import json
import pandas as pd
import ssl

ServiceKey=""

In [111]:
#[CODE 1]
# url 을 받으면 요청 , OK 면 json 반환
context = ssl.create_default_context()
context.set_ciphers('DEFAULT:@SECLEVEL=1')
def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req, context=context)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [112]:
def getTourismStatsItem(yyyy, city_code, county_code):
    service_url = "https://apis.data.go.kr/B552061/frequentzoneBicycle/getRestFrequentzoneBicycle"
    # type , serviceKey
    prameters = "?type=json&ServiceKey=" + ServiceKey
    prameters += "&searchYearCd=" + yyyy
    prameters += "&siDo=" + city_code
    prameters += "&guGun=" + county_code
    prameters += "&numOfRows=100"
    prameters += "&pageNo=1"
    url = service_url + prameters
    print(url)
    responseDecode = getRequestUrl(url)
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [113]:
def getTourismStatsService(city_cd,county_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    spot_nm=''
    dataEND = 0

    for year in range(nStartYear, nEndYear+1):
        yyyy = "{0}".format(str(year))
        jsonData = getTourismStatsItem(yyyy,city_cd,county_cd) #[CODE 2]
        if (jsonData['resultCode'] == '00'):
        # 아직 입력된 범위까지 수집하지 않았지만, 더이상 제공되는 데이터가 없는 경우 -------------------
            if len(jsonData['items']['item']) == 0:
                dataEND = "{0}".format(str(year-1))
                break

            #반환할 데이터 항목을 추출하여 저장
            for item in jsonData['items']['item']:
                city_county_name = item['sido_sgg_nm']
                city_county_name = city_county_name.replace(' ', '')
                afos_fid = item['afos_fid']
                spot_nm = item['spot_nm']
                occrrnc_cnt = item['occrrnc_cnt']
                print('[ %s : %s_%s ]' %(yyyy, city_county_name, spot_nm))
                print('-'*70)
                jsonResult.append({'city_county_name': city_county_name,'spot_nm':spot_nm, 'afos_fid': afos_fid, 'year': yyyy, 'occrrnc_cnt': occrrnc_cnt})
                result.append([city_county_name, spot_nm, afos_fid, yyyy, occrrnc_cnt])

    #마지막 jsonData를 확인용으로 출력하기......................................................
    print ('<마지막 jsonData>\n', json.dumps(jsonData, indent=4, sort_keys=True, ensure_ascii=False))

    return (jsonResult, result, city_cd, county_cd, spot_nm, dataEND)

In [114]:
def main():
    jsonResult = []
    result = []
    natName=''

    print("<< 자전거 사고 다발지역 수집 >>")
    county_cd = 0
    city_cd = input('시도 코드를 입력하세요(서울: 11 / 부산: 26) : ')
    if (city_cd == '11'):
        county_cd = input('구군 코드를 입력하세요(강북구: 305 / 강서구: 500 / 관악구: 620)')
    if (city_cd == '26'):
        county_cd = input('구군 코드를 입력하세요(강서구: 4)')
    nStartYear =int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))

    jsonResult, result, city_cd, county_cd, spot_nm, dataEND = \
                getTourismStatsService(city_cd, county_cd, nStartYear, nEndYear) #[CODE 3]

    if (spot_nm=='') : #URL 요청은 성공하였지만, 데이터 제공이 안된 경우
        print('데이터가 전달되지 않았습니다. 공공데이터포털의 서비스 상태를 확인하기 바랍니다.')
    else:
        #파일저장 1 : json 파일
        with open('./%s_%s_%d_%s.json' % (city_cd, county_cd, nStartYear, dataEND), 'w',
                    encoding='utf8') as outfile:
            jsonFile  = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
            outfile.write(jsonFile)
        #파일저장 2 : csv 파일
        columns = ["사고다발지역 주소", "사고다발지역 이름", "위치 정보", "해당연도", "사고건수"]
        result_df = pd.DataFrame(result, columns = columns)
        result_df.to_csv('./%s_%s_%d_%s.csv' % (city_cd, county_cd, nStartYear, dataEND),
                    index=False, encoding='utf-8')

if __name__ == '__main__':
    main()

<< 자전거 사고 다발지역 수집 >>
https://apis.data.go.kr/B552061/frequentzoneBicycle/getRestFrequentzoneBicycle?type=json&ServiceKey=zytXafKy%2Bm%2FrnUWmID8dWjMYtcsoEGBLO42CK%2BGCj7GqtS2rAskQyRttpfZ2dvx8SxomL8i90ZMPjBEJM2Plig%3D%3D&searchYearCd=2020&siDo=11&guGun=500&numOfRows=100&pageNo=1
[2025-02-19 17:10:03.957698] Url Request Success
[ 2020 : 서울특별시강서구1_서울특별시 강서구 화곡동(조흥은행등촌동지점 부근) ]
----------------------------------------------------------------------
[ 2020 : 서울특별시강서구2_서울특별시 강서구 내발산동(발산역앞교차로 부근) ]
----------------------------------------------------------------------
https://apis.data.go.kr/B552061/frequentzoneBicycle/getRestFrequentzoneBicycle?type=json&ServiceKey=zytXafKy%2Bm%2FrnUWmID8dWjMYtcsoEGBLO42CK%2BGCj7GqtS2rAskQyRttpfZ2dvx8SxomL8i90ZMPjBEJM2Plig%3D%3D&searchYearCd=2021&siDo=11&guGun=500&numOfRows=100&pageNo=1
[2025-02-19 17:10:04.227243] Url Request Success
[ 2021 : 서울특별시강서구1_서울특별시 강서구 화곡동(동서파크빌라 부근) ]
----------------------------------------------------------------------
[ 2021 : 